# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


# Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [3]:
data = pd.read_csv('cost_revenue_dirty.csv')

# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [22]:
# %timeit data.info()
# %timeit data.info(show_counts=True)
# %timeit data[data.isna().any(axis=1)]
# # %timeit data[data.isna().any(axis=None)]
# %timeit data.isna().any().any()
# axis=None did not work. The two .info methods were equivalent, as were the .isna() methods 


In [26]:
print(data.shape)
print(data.info())
# print(data.nunique())
print(f'Any nan? {data[data.isna().any(axis=1)]}')
print(f"Duplicates: \n{data[data.duplicated(keep=False, subset=['Release_Date', 'Movie_Title'])]}")
data.sample(5)

(5391, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB
None
Any nan? Empty DataFrame
Columns: [Rank, Release_Date, Movie_Title, USD_Production_Budget, USD_Worldwide_Gross, USD_Domestic_Gross]
Index: []
Duplicates: 
      Rank Release_Date        Movie_Title USD_Production_Budget  \
3054  5289   12/31/2007  A Dog's Breakfast              $120,000   
3055  5290   12/31/2007  A Dog's Breakfast              $120,000   

     USD_Worldwide_Gross USD_Domestic_Gross  
3054         

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
3747,3227,10/2/2010,Triangle,"$12,000,000","$1,595,168",$0
62,3999,1/1/1954,The Egyptian,"$5,000,000","$15,000,000","$15,000,000"
2784,983,10/20/2006,Flags of Our Fathers,"$53,000,000","$63,657,941","$33,602,376"
3264,2582,11/4/2008,Prom Night,"$18,000,000","$57,193,655","$43,869,350"
2405,4109,4/3/2005,Dear Frankie,"$4,600,000","$3,099,369","$1,340,891"


### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [27]:
data.USD_Domestic_Gross = data.USD_Domestic_Gross.str.replace(pat=r'\D', repl='', regex=True).astype(int)
data.USD_Production_Budget = data.USD_Production_Budget.str.replace(pat=r'\D', repl='', regex=True).astype(int)
data.USD_Worldwide_Gross = data.USD_Worldwide_Gross.str.replace(pat=r'\D', repl='', regex=True).astype(int)
data.sample(5)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
646,4051,9/21/1992,Light Sleeper,5000000,1050861,1050861
1547,2029,12/15/2000,Chocolat,25000000,152500343,71309760
2783,5148,10/18/2006,51 Birch Street,350000,84689,84689
3573,5015,1/26/2010,B-Girl,700000,1160,0
2285,392,9/4/2004,The Alamo,92000000,23911362,22406362


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   int64 
 4   USD_Worldwide_Gross    5391 non-null   int64 
 5   USD_Domestic_Gross     5391 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 252.8+ KB


**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [29]:
data.Release_Date = pd.to_datetime(data.Release_Date)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Rank                   5391 non-null   int64         
 1   Release_Date           5391 non-null   datetime64[ns]
 2   Movie_Title            5391 non-null   object        
 3   USD_Production_Budget  5391 non-null   int64         
 4   USD_Worldwide_Gross    5391 non-null   int64         
 5   USD_Domestic_Gross     5391 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 252.8+ KB


In [30]:
# Did not drop the duplicate above since it was not part of the challenge. 
# Will do it here.
data = data.drop_duplicates(subset=['Release_Date', 'Movie_Title'])
print(f"Duplicates: \n{data[data.duplicated(keep=False, subset=['Release_Date', 'Movie_Title'])]}")
data.info()

Duplicates: 
Empty DataFrame
Columns: [Rank, Release_Date, Movie_Title, USD_Production_Budget, USD_Worldwide_Gross, USD_Domestic_Gross]
Index: []
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5390 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Rank                   5390 non-null   int64         
 1   Release_Date           5390 non-null   datetime64[ns]
 2   Movie_Title            5390 non-null   object        
 3   USD_Production_Budget  5390 non-null   int64         
 4   USD_Worldwide_Gross    5390 non-null   int64         
 5   USD_Domestic_Gross     5390 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 294.8+ KB


### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [39]:
print(f'Avg Prod Budget: {data.USD_Production_Budget.mean()}')
print(f'Avg Worldwide Gross Rev: {data.USD_Worldwide_Gross.mean()}')
# Getting too bored to type all these out. I think describe() gives us all of the above.
print(data.describe())
# I think the only question not answered by describe() is #6
# #6 Lowest
print(f'Lowest Budget Film:')
data[data.USD_Production_Budget == data.USD_Production_Budget.min()]


Avg Prod Budget: 31119487.807977736
Avg Worldwide Gross Rev: 88871907.19870129
          Rank  USD_Production_Budget  USD_Worldwide_Gross  USD_Domestic_Gross
count 5,390.00               5,390.00             5,390.00            5,390.00
mean  2,695.52          31,119,487.81        88,871,907.20       41,243,169.82
std   1,556.14          40,525,356.93       168,469,037.10       66,033,082.74
min       1.00               1,100.00                 0.00                0.00
25%   1,348.25           5,000,000.00         3,877,146.00        1,335,065.75
50%   2,695.50          17,000,000.00        27,460,037.00       17,196,565.00
75%   4,042.75          40,000,000.00        96,461,821.00       52,348,661.50
max   5,391.00         425,000,000.00     2,783,918,982.00      936,662,225.00
Lowest Budget Film:


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
2427,5391,2005-05-08,My Date With Drew,1100,181041,181041


In [40]:
# 6 Highest
print(f'Highest Budget Film:')
data[data.USD_Production_Budget == data.USD_Production_Budget.max()]

Highest Budget Film:


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
3529,1,2009-12-18,Avatar,425000000,2783918982,760507625


# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

In [41]:
data[data.USD_Domestic_Gross == 0].sort_values(by='USD_Production_Budget', ascending=False)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
5388,96,2020-12-31,Singularity,175000000,0,0
5387,126,2018-12-18,Aquaman,160000000,0,0
5384,321,2018-09-03,A Wrinkle in Time,103000000,0,0
5385,366,2018-10-08,Amusement Park,100000000,0,0
5090,556,2015-12-31,"Don Gato, el inicio de la pandilla",80000000,4547660,0
...,...,...,...,...,...,...
4787,5371,2014-12-31,Stories of Our Lives,15000,0,0
3056,5374,2007-12-31,Tin Can Man,12000,0,0
4907,5381,2015-05-19,Family Motocross,10000,0,0
5006,5389,2015-09-29,Signed Sealed Delivered,5000,0,0


**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

In [42]:
data[data.USD_Worldwide_Gross == 0].sort_values(by='USD_Production_Budget', ascending=False)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
5388,96,2020-12-31,Singularity,175000000,0,0
5387,126,2018-12-18,Aquaman,160000000,0,0
5384,321,2018-09-03,A Wrinkle in Time,103000000,0,0
5385,366,2018-10-08,Amusement Park,100000000,0,0
5338,879,2017-04-08,The Dark Tower,60000000,0,0
...,...,...,...,...,...,...
4787,5371,2014-12-31,Stories of Our Lives,15000,0,0
3056,5374,2007-12-31,Tin Can Man,12000,0,0
4907,5381,2015-05-19,Family Motocross,10000,0,0
5006,5389,2015-09-29,Signed Sealed Delivered,5000,0,0


### Filtering on Multiple Conditions

In [44]:
intl_rels = data.loc[(data.USD_Worldwide_Gross != 0) &
                     (data.USD_Domestic_Gross == 0)]
print(f'Number of International-Only Releases: {len(intl_rels)}')
intl_rels.sample(5)

Number of International-Only Releases: 155


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
5322,2946,2017-02-17,In Dubious Battle,15000000,4892,0
3414,4074,2009-07-07,Flying By,5000000,4568257,0
4694,2945,2014-10-10,Automata,15000000,29050,0
4012,4158,2011-10-21,Cargo,4200000,313230,0
2409,4747,2005-04-11,Wal-Mart: The High Cost of Low Price,1500000,58692,0


**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

In [46]:
intl_only = data.query(expr='(USD_Worldwide_Gross != 0) and (USD_Domestic_Gross == 0)')
print(f'Number of International-Only Releases: {len(intl_only)}')
intl_only.sample(5)

Number of International-Only Releases: 155


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
5340,1506,2017-04-14,Queen of the Desert,36000000,1480089,0
3899,4780,2011-05-24,Forget Me Not,1300000,13465,0
2155,4236,2003-12-31,Nothing,4000000,63180,0
5271,1433,2016-11-11,USS Indianapolis: Men of Courage,40000000,1079040,0
5174,2649,2016-04-15,I Am Wrath,18000000,6784,0


### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [4]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 